In [1]:
class NodeInfo:
    def __init__(self, p1_dice, p2_dice, p1_bid_strat, p1_cha_strat, p2_bid_strat, p2_cha_strat, start_player, start_bid_strat):
        self.p1_dice = p1_dice
        self.p2_dice = p2_dice
        self.p1_bid_strat = p1_bid_strat
        self.p1_cha_strat = p1_cha_strat
        self.p2_bid_strat = p2_bid_strat
        self.p2_cha_strat = p2_cha_strat
        self.start_player = start_player
        self.start_bid_strat = start_bid_strat
        
        self.model_spec = model_spec = f"ld_{p1_dice}-{p2_dice}_{p1_bid_strat}_{p1_cha_strat}_{p2_bid_strat}_{p2_cha_strat}_{start_player}_{start_bid_strat}"
        self.left = None
        self.right = None
        self.p1_win_round = self.get_model_checking()
        self.p1_win_game = None
        
    def __str__(self):
        return f"{self.p1_dice}/{self.p2_dice}, P{self.start_player}"
    
    def get_winner(self):
        if self.p1_dice == 0:
            return 2
        if self.p2_dice == 0:
            return 1
        return 0
    
    def get_model_checking(self):
        # construct a model spec, and read in the results for this particular model. If no results exist yet for this model,
        # generate them!
        
        if self.get_winner() != 0:
            return self.get_winner()
        else:
            data_location = f"LiarsDice/data/{self.model_spec}/{self.model_spec}_p1_win_round.csv"
            try:
                with open(data_location) as f:
                    lines = f.readlines()
                    return(float(lines[1]))
            except FileNotFoundError:
                print(f"Need model checking for {self.model_spec}")
                return 0
    
print(NodeInfo(2, 2, 0, 0, 1, 1, 1, 1).p1_win_round)
print((NodeInfo(3, 0, 1, 1, 0, 0, 1, 1)).get_winner())
print((NodeInfo(0, 3, 1, 1, 0, 0, 1, 1)).get_winner())

0.5
1
2


In [2]:
def game_tree(root):
    # given the starting configuration, returns the tree for a game of Liar's dice
    
    if root.get_winner():
        return root
    else:
        p1_lose = NodeInfo(root.p1_dice - 1, root.p2_dice, root.p1_bid_strat, root.p1_cha_strat, root.p2_bid_strat, root.p2_cha_strat, 1, root.start_bid_strat)
        root.left = game_tree(p1_lose)
        
        p2_lose = NodeInfo(root.p1_dice, root.p2_dice - 1, root.p1_bid_strat, root.p1_cha_strat, root.p2_bid_strat, root.p2_cha_strat, 2, root.start_bid_strat)
        root.right = game_tree(p2_lose)
        
        return root

game_22 = game_tree(NodeInfo(2, 2, 0, 0, 1, 1, 1, 1))

In [3]:
def prob_p1_wins(root):
    # given a starting configuration, calculates the probability that p1 wins the overall game.
    
    if root.get_winner() == 1:
        return 1
    if root.get_winner() == 2:
        return 0
    
    if root.p1_win_round is None:
        print(f"Need model checking results for {root}")
        root.p1_win_round = 1
    
    return  ((1 - root.p1_win_round) * prob_p1_wins(root.left) + root.p1_win_round * prob_p1_wins(root.right))

print(prob_p1_wins(game_22))

0.7469135833333334


In [4]:
prob_p1_wins(game_tree(NodeInfo(1, 1, 2)))

TypeError: __init__() missing 5 required positional arguments: 'p1_cha_strat', 'p2_bid_strat', 'p2_cha_strat', 'start_player', and 'start_bid_strat'

In [64]:
build_model(2, 2, 0.5, 0.75, 0, 0, 1, 1, 1, 0)

ld_2-2_0_0_1_1_1_0
LiarsDice/models/ld_2-2_0_0_1_1_1_0/ld_2-2_0_0_1_1_1_0.prism
Reached this point!!!!


The syntax of the command is incorrect.


Reached this point!!!


Traceback (most recent call last):
  File "utils/pyprism.py", line 79, in <module>
    with args.input.open('r') as f_in, args.output.open('w') as f_out:
  File "C:\Users\Lewis\Anaconda3\lib\pathlib.py", line 1186, in open
    opener=self._opener)
  File "C:\Users\Lewis\Anaconda3\lib\pathlib.py", line 1039, in _opener
    return self._accessor.open(self, flags, mode)
FileNotFoundError: [Errno 2] No such file or directory: 'LiarsDice\\models\\ld_2-2_0_0_1_1_1_0\\ld_2-2_0_0_1_1_1_0.prism'
Error: A JNI error has occurred, please check your installation and try again
Exception in thread "main" java.lang.UnsupportedClassVersionError: prism/PrismCL has been compiled by a more recent version of the Java Runtime (class file version 55.0), this version of the Java Runtime only recognizes class file versions up to 52.0
	at java.lang.ClassLoader.defineClass1(Native Method)
	at java.lang.ClassLoader.defineClass(Unknown Source)
	at java.security.SecureClassLoader.defineClass(Unknown Source)
	at jav

In [63]:
def build_model(p1_dice, p2_dice, p1_c, p2_c, p1_bid_strat, p1_cha_strat, p2_bid_strat, p2_cha_strat, starting_player, start_bid_strat):

    model_spec = f"ld_{p1_dice}-{p2_dice}_{p1_bid_strat}_{p1_cha_strat}_{p2_bid_strat}_{p2_cha_strat}_{starting_player}_{start_bid_strat}"
    
    print(model_spec)
    model_location = "LiarsDice/models/" + model_spec
    !mkdir $model_location

    output_model = f"LiarsDice/models/{model_spec}/{model_spec}.prism"
    
    print(output_model)

    write_params(p1_dice, p2_dice, p1_c, p2_c, p1_bid_strat, p1_cha_strat, p2_bid_strat, p2_cha_strat, starting_player, start_bid_strat)
 
    print("Reached this point!!!!")
    !python utils/pyprism.py LiarsDice/LiarsDice.pyprism $output_model LiarsDice/params/{model_spec}.csv

    print("Reached this point!!!")
    props_file = "LiarsDice/LD_manual.props"

    !prism-ext {output_model} $props_file -prop 5 -const p1_c={p1_c},p2_c={p2_c} -gridresolution 1 -exportresults LiarsDice/data/{model_spec}/{model_spec}_p1_win_round.csv:csv

In [37]:
def write_params(p1_dice, p2_dice, p1_c, p2_c, p1_bid_strat, p1_cha_strat, p2_bid_strat, p2_cha_strat, starting_player, start_bid_strat):
    model_spec = f"ld_{p1_dice}-{p2_dice}_{p1_bid_strat}_{p1_cha_strat}_{p2_bid_strat}_{p2_cha_strat}_{starting_player}_{start_bid_strat}"
    
    with open(f"LiarsDice/params/{model_spec}.csv", "w") as params:
        params.write(f"p1_die, {p1_dice}\n")
        params.write(f"p1_bid_strat, {p1_bid_strat}\n")
        params.write(f"p1_cha_strat, {p1_cha_strat}\n")
        params.write(f"p2_die, {p2_dice}\n")
        params.write(f"p2_bid_strat, {p2_bid_strat}\n")
        params.write(f"p2_cha_strat, {p2_cha_strat}\n")
        params.write(f"start_player, {starting_player}\n")
        params.write(f"start_bid_strat, {start_bid_strat}\n")
        params.write("p1, 1\n")
        params.write("p2, 2")

In [65]:
model_spec = f"ld_{p1_dice}-{p2_dice}_{p1_bid_strat}_{p1_cha_strat}_{p2_bid_strat}_{p2_cha_strat}_{starting_player}_{start_bid_strat}"
    
output_model = f"LiarsDice/models/{model_spec}/{model_spec}.prism"

!python utils/pyprism.py LiarsDice/LiarsDice.pyprism $output_model LiarsDice/params/{model_spec}.csv

props_file = "LiarsDice/LD_manual.props"
    
!mkdir "LiarsDice/data/$model_spec"

!prism-ext {output_model} $props_file -prop 5 -const p2_c=0.5 -gridresolution 1 -exportresults LiarsDice/data/{model_spec}/{model_spec}_p1_win_round.csv:csv

NameError: name 'p1_dice' is not defined

In [70]:
!prism-ext

Error: A JNI error has occurred, please check your installation and try again
Exception in thread "main" java.lang.UnsupportedClassVersionError: prism/PrismCL has been compiled by a more recent version of the Java Runtime (class file version 55.0), this version of the Java Runtime only recognizes class file versions up to 52.0
	at java.lang.ClassLoader.defineClass1(Native Method)
	at java.lang.ClassLoader.defineClass(Unknown Source)
	at java.security.SecureClassLoader.defineClass(Unknown Source)
	at java.net.URLClassLoader.defineClass(Unknown Source)
	at java.net.URLClassLoader.access$100(Unknown Source)
	at java.net.URLClassLoader$1.run(Unknown Source)
	at java.net.URLClassLoader$1.run(Unknown Source)
	at java.security.AccessController.doPrivileged(Native Method)
	at java.net.URLClassLoader.findClass(Unknown Source)
	at java.lang.ClassLoader.loadClass(Unknown Source)
	at sun.misc.Launcher$AppClassLoader.loadClass(Unknown Source)
	at java.lang.ClassLoader.loadClass(Unknown Source)
	at 

In [69]:
!java -version

java version "1.8.0_281"
Java(TM) SE Runtime Environment (build 1.8.0_281-b09)
Java HotSpot(TM) 64-Bit Server VM (build 25.281-b09, mixed mode)
